In [1]:
import sys
import os
import json

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
SRC_PATH = os.path.join(PROJECT_ROOT, 'src')
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Đã thêm '{SRC_PATH}' vào sys.path")
else:
    print(f"'{SRC_PATH}' đã có trong sys.path")

Đã thêm 'd:\BookInsight\src' vào sys.path


In [2]:
try:
    from agent.agent import create_book_agent_executor
    print("Import 'create_book_agent_executor' thành công")
except ImportError as e:
    print(f"Looi Import:{e}")
    print("Kiểm tra lại 1. 'Sources Root 2. Các file __init__.py")
except Exception as e:
    print(f"Lỗi không mong muốn: {e}")

# Khởi tạo Agent Executor (load model, DB, index....)
print("\n Đang khởi tạo Factory để tạo Agent Executor...")
try: 
    agent_executor = create_book_agent_executor()
    print("Agent executor đã sẵn sàng")
except Exception as e:
    print(f"Lỗi khi khởi tạo Factory :{type(e)} - {e}")


t
Import 'create_book_agent_executor' thành công

 Đang khởi tạo Factory để tạo Agent Executor...
[AgentFactory] Đang tạo Agent Executor...
[AgentFactory] Đang khởi tạo các động cơ (RAG, SQL)...
[SmartRetriever] Đang khởi tạo các thành phần...
[VectorStore] Đang khởi tạo...
[VectorStore] Đang tải Model Embedding: BAAI/bge-small-en-v1.5
[VectorStore] Đang tải FAISS index từ: d:\BookInsight\data\vectorstores\index.faiss
[VectorStore] Đang tải metadata từ: d:\BookInsight\data\vectorstores\unique_ids.pkl
 [VectorStore] Khởi tạo hoàn tất. Sẵn sàng tìm kiếm.
[SQLDatabase] Đang kết nối tới: d:\BookInsight\data\database\books.db
[SQLDatabase] Kết nối thành công.
[QueryExpander] Đã khởi tạo với model: gpt-3.5-turbo
[SmartRetriever] Khởi tạo hoàn tất.
[AgentFactory] Đang khởi tạo 4 Tools...
[AgentFactory] Đang lắp ráp Agent...
[AgentFactory] Agent Executor đã được tạo thành công.
Agent executor đã sẵn sàng


d:\BookInsight\src\agent\agent.py:76: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
import sqlite3
import os

try:
    from config import SQL_DB_PATH
except ImportError:
    # (Dùng lại path thủ công nếu import lỗi)
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
    SQL_DB_PATH = os.path.join(PROJECT_ROOT, "data", "database", "books.db")

print(f"--- Đang dọn dẹp 'user_preferences' trong {SQL_DB_PATH} ---")
try:
    conn = sqlite3.connect(SQL_DB_PATH)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM user_preferences;")
    conn.commit()
    conn.close()
    print("Đã xóa tất cả sở thích cũ. Bảng 'user_preferences' đã sạch.")
except Exception as e:
    print(f"Lỗi khi dọn dẹp bảng: {e}")

--- Đang dọn dẹp 'user_preferences' trong d:\BookInsight\data\database\books.db ---
Đã xóa tất cả sở thích cũ. Bảng 'user_preferences' đã sạch.


In [5]:
def chat(question):
    print(f"\n 👤 User: {question}")
    try: 
        response = agent_executor.invoke({"input": question})
        print("\n 🤖 Agent (Final Answer):")
        print(response['output'])
    except Exception as e:
        print(f"Lỗi khi chạy Agent: {e}")

# test 1: test memory và tool 3 (save)
print("\nBắt đầu lưu SỞ THÍCH")
chat("I like books about 'Detective fiction'")

#test 2: test tool 4 (get Rec)
print("\nBắt đầu test gợi ý cá nhân hóa")
chat("Can you recommend a good book for me?")

# test 3: tool 1 (RAG) & tool 2 SQL
print("\n Bắt đầu test RAG + SQL")
chat("What is the highest-rated book about 'Mystery novel' can you summarize it?")



Bắt đầu lưu SỞ THÍCH

 👤 User: I like books about 'Detective fiction'


> Entering new AgentExecutor chain...

Invoking: `save_user_preference` with `{'preference_type': 'category', 'preference_value': 'Detective fiction'}`



[SavePreferenceTool] Đang lưu: category = Detective fiction
Đã lưu thành công sở thích :category = Detective fiction.
Invoking: `get_personalized_recommendation` with `{}`



[GetRecommendationTool] ... Đang tìm sở thích đã lưu...
[GetRecommendationTool] Đã tạo query cá nhân hóa: 'Tìm sách dựa trên các sở thích sau: category Detective fiction'
[GetRecommendationTool] Đang gọi SmartRAGTool để tìm sách...

[SmartRAGTool] đang chạy với câu hỏi: Tìm sách dựa trên các sở thích sau: category Detective fiction

[SmartRetriever] Bắt đầu truy vấn cho: 'Tìm sách dựa trên các sở thích sau: category Detective fiction'
[QueryExpander] Đang mở rộng câu hỏi: 'Tìm sách dựa trên các sở thích sau: category Detective fiction'...
-> [QueryExpander] Các câu hỏi đã mở rộng: ['Tìm sác